In [21]:
import pandas as pd
import numpy as np
import re
import pip
pip.main(["install", "openpyxl"])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


  Using cached openpyxl-3.0.10-py2.py3-none-any.whl (242 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


0

In [49]:
sample_df = pd.read_excel("Silver Samples 4712.xlsx")
sample_df.head()

,TrialID,PMID,FundingSource_Text,Date_registration_Text,Source_Register_Text,Inclusion_agemin,Inclusion_agemax,Inclusion_gender,Date_enrollement_Text,Target_size,...,ComplianceWeeks,ComplianceMonths,ComplianceGroup,FDAComplianceGroup,NHSComplianceGroup,RegToSubDuration,SubToPubDuration,Countries_UK,Countries_US,Countries.1
0,NCT01145820,28866140,University of Birmingham,Jun 16 2010 12:00AM,ClinicalTrials.gov,18 Years,60 Years,Both,Jun 1 2010 12:00AM,246,...,-2,0,3WeeksLate,Yes,Yes,3Years+,12Months+,UK,na,United Kingdom
1,NCT01258829,29778861,Imperial College London,Dec 10 2010 12:00AM,ClinicalTrials.gov,18 Years,NaN,Both,Dec 1 2010 12:00AM,403,...,-1,0,3WeeksLate,Yes,Yes,3Years+,6-12Months,UK,na,United Kingdom
2,NCT02612415,29866165,Great Ormond Street Hospital for Children NHS ...,Nov 20 2015 12:00AM,ClinicalTrials.gov,36 Weeks,15 Years,Both,Nov 1 2015 12:00AM,121,...,-2,0,3WeeksLate,Yes,Yes,2-3Years,Within6Months,UK,na,United Kingdom
3,NCT01971437,28439633,Medway NHS Foundation Trust,Oct 21 2013 12:00AM,ClinicalTrials.gov,18 Years,NaN,Female,"October 1, 2013",70,...,-2,0,3WeeksLate,Yes,Yes,3Years+,6-12Months,UK,na,United Kingdom
4,NCT01879813,30172995,University of Leeds,Oct 15 2012 12:00AM,ClinicalTrials.gov,18 Years,55 Years,Male,"October 1, 2012",54,...,-2,0,3WeeksLate,Yes,Yes,3Years+,6-12Months,UK,na,United Kingdom


In [52]:
# Option 1 - Convert all ages to years
def convert_to_years(age_string):
    # Get digit out
    digits = re.findall("\d*", age_string)[0]

    if "Year" in age_string:
        return float(digits)
    if "Week" in age_string:
        return float(digits) / 52
    if "Day" in age_string:
        return float(digits) / 365
    if "Hour" in age_string:
        return float(digits) / (365 * 24)
    else:
        return np.nan
        

In [53]:
# Process data
sample_df['Inclusion_agemax1'] = sample_df['Inclusion_agemax'].fillna("N/A").apply(convert_to_years)
sample_df['Inclusion_agemin1'] = sample_df['Inclusion_agemin'].fillna("N/A").apply(convert_to_years)
sample_df.head()

,TrialID,PMID,FundingSource_Text,Date_registration_Text,Source_Register_Text,Inclusion_agemin,Inclusion_agemax,Inclusion_gender,Date_enrollement_Text,Target_size,...,ComplianceGroup,FDAComplianceGroup,NHSComplianceGroup,RegToSubDuration,SubToPubDuration,Countries_UK,Countries_US,Countries.1,Inclusion_agemax1,Inclusion_agemin1
0,NCT01145820,28866140,University of Birmingham,Jun 16 2010 12:00AM,ClinicalTrials.gov,18 Years,60 Years,Both,Jun 1 2010 12:00AM,246,...,3WeeksLate,Yes,Yes,3Years+,12Months+,UK,na,United Kingdom,60.0,18.000000
1,NCT01258829,29778861,Imperial College London,Dec 10 2010 12:00AM,ClinicalTrials.gov,18 Years,NaN,Both,Dec 1 2010 12:00AM,403,...,3WeeksLate,Yes,Yes,3Years+,6-12Months,UK,na,United Kingdom,NaN,18.000000
2,NCT02612415,29866165,Great Ormond Street Hospital for Children NHS ...,Nov 20 2015 12:00AM,ClinicalTrials.gov,36 Weeks,15 Years,Both,Nov 1 2015 12:00AM,121,...,3WeeksLate,Yes,Yes,2-3Years,Within6Months,UK,na,United Kingdom,15.0,0.692308
3,NCT01971437,28439633,Medway NHS Foundation Trust,Oct 21 2013 12:00AM,ClinicalTrials.gov,18 Years,NaN,Female,"October 1, 2013",70,...,3WeeksLate,Yes,Yes,3Years+,6-12Months,UK,na,United Kingdom,NaN,18.000000
4,NCT01879813,30172995,University of Leeds,Oct 15 2012 12:00AM,ClinicalTrials.gov,18 Years,55 Years,Male,"October 1, 2012",54,...,3WeeksLate,Yes,Yes,3Years+,6-12Months,UK,na,United Kingdom,55.0,18.000000


In [74]:
# Option 2 - Break up measurement and unit
sample_df[['Inclusion_agemax2', 'Inclusion_agemax2_units']] = sample_df['Inclusion_agemax'].fillna('N/A N/A').str.split(expand=True)
sample_df[['Inclusion_agemin2', 'Inclusion_agemin2_units']] = sample_df['Inclusion_agemin'].fillna('N/A N/A').str.split(expand=True)
sample_df.head()

# There are some cases where the units diff between max and min (for example, min is in weeks, max is in years). This means we should likely keep the two separte unit columns.
sample_df[sample_df['Inclusion_agemax2_units'] != sample_df['Inclusion_agemin2_units']]

,TrialID,PMID,FundingSource_Text,Date_registration_Text,Source_Register_Text,Inclusion_agemin,Inclusion_agemax,Inclusion_gender,Date_enrollement_Text,Target_size,...,SubToPubDuration,Countries_UK,Countries_US,Countries.1,Inclusion_agemax1,Inclusion_agemin1,Inclusion_agemax2,Inclusion_agemax2_units,Inclusion_agemin2,Inclusion_agemin2_units
1,NCT01258829,29778861,Imperial College London,Dec 10 2010 12:00AM,ClinicalTrials.gov,18 Years,NaN,Both,Dec 1 2010 12:00AM,403,...,6-12Months,UK,na,United Kingdom,NaN,18.000000,N/A,N/A,18,Years
2,NCT02612415,29866165,Great Ormond Street Hospital for Children NHS ...,Nov 20 2015 12:00AM,ClinicalTrials.gov,36 Weeks,15 Years,Both,Nov 1 2015 12:00AM,121,...,Within6Months,UK,na,United Kingdom,15.000000,0.692308,15,Years,36,Weeks
3,NCT01971437,28439633,Medway NHS Foundation Trust,Oct 21 2013 12:00AM,ClinicalTrials.gov,18 Years,NaN,Female,"October 1, 2013",70,...,6-12Months,UK,na,United Kingdom,NaN,18.000000,N/A,N/A,18,Years
6,NCT03176433,29670285,University of Oxford,May 4 2017 12:00AM,ClinicalTrials.gov,18 Years,NaN,Both,"May 1, 2017",30,...,6-12Months,UK,na,United Kingdom,NaN,18.000000,N/A,N/A,18,Years
8,NCT02505503,30265963,York Teaching Hospitals NHS Foundation Trust,Jul 21 2015 12:00AM,ClinicalTrials.gov,16 Years,NaN,Both,Jul 1 2015 12:00AM,38,...,Within6Months,UK,na,United Kingdom,NaN,16.000000,N/A,N/A,16,Years
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4706,NCT02510144,30054245,University of Maryland,Jul 23 2015 12:00AM,ClinicalTrials.gov,18 Years,NaN,Both,"January 1, 2016",80,...,Within6Months,na,US,United States,NaN,18.000000,N/A,N/A,18,Years
4708,NCT02244918,29787858,University of Kansas Medical Center,Sep 17 2014 12:00AM,ClinicalTrials.gov,18 Years,NaN,Both,May 1 2015 12:00AM,278,...,6-12Months,na,US,United States,NaN,18.000000,N/A,N/A,18,Years
4709,NCT02313181,29550235,"University of California, San Francisco",Dec 5 2014 12:00AM,ClinicalTrials.gov,NaN,72 Hours,Both,Jan 7 2015 12:00AM,328,...,6-12Months,na,US,United States,0.008219,NaN,72,Hours,N/A,N/A
4710,NCT02205307,29751360,Novadaq Technologies Inc.,Jul 29 2014 12:00AM,ClinicalTrials.gov,18 Years,NaN,Both,"January 1, 2015",330,...,6-12Months,na,US,United States,NaN,18.000000,N/A,N/A,18,Years


In [79]:
# Option 3 - Age Category
def age_category(age_string):
    if not pd.notna(age_string):
        return np.nan
    elif "Week" in age_string or "Hour" in age_string or "Day" in age_string:
        return "Infant"
    elif float(age_string.split()[0]) < 18:
        return "Child"
    else:
        return "Adult"

In [80]:
sample_df['Inclusion_agemax_category'] = sample_df['Inclusion_agemax'].apply(age_category)
sample_df['Inclusion_agemin_category'] = sample_df['Inclusion_agemin'].apply(age_category)
sample_df.head()

,TrialID,PMID,FundingSource_Text,Date_registration_Text,Source_Register_Text,Inclusion_agemin,Inclusion_agemax,Inclusion_gender,Date_enrollement_Text,Target_size,...,Countries_US,Countries.1,Inclusion_agemax1,Inclusion_agemin1,Inclusion_agemax2,Inclusion_agemax2_units,Inclusion_agemin2,Inclusion_agemin2_units,Inclusion_agemax_category,Inclusion_agemin_category
0,NCT01145820,28866140,University of Birmingham,Jun 16 2010 12:00AM,ClinicalTrials.gov,18 Years,60 Years,Both,Jun 1 2010 12:00AM,246,...,na,United Kingdom,60.0,18.000000,60,Years,18,Years,Adult,Adult
1,NCT01258829,29778861,Imperial College London,Dec 10 2010 12:00AM,ClinicalTrials.gov,18 Years,NaN,Both,Dec 1 2010 12:00AM,403,...,na,United Kingdom,NaN,18.000000,N/A,N/A,18,Years,NaN,Adult
2,NCT02612415,29866165,Great Ormond Street Hospital for Children NHS ...,Nov 20 2015 12:00AM,ClinicalTrials.gov,36 Weeks,15 Years,Both,Nov 1 2015 12:00AM,121,...,na,United Kingdom,15.0,0.692308,15,Years,36,Weeks,Child,Infant
3,NCT01971437,28439633,Medway NHS Foundation Trust,Oct 21 2013 12:00AM,ClinicalTrials.gov,18 Years,NaN,Female,"October 1, 2013",70,...,na,United Kingdom,NaN,18.000000,N/A,N/A,18,Years,NaN,Adult
4,NCT01879813,30172995,University of Leeds,Oct 15 2012 12:00AM,ClinicalTrials.gov,18 Years,55 Years,Male,"October 1, 2012",54,...,na,United Kingdom,55.0,18.000000,55,Years,18,Years,Adult,Adult
